In [1]:
!which python
!python --version
!pip show torch

/root/miniconda3/envs/deepseek/bin/python
Python 3.13.7
Name: torch
Version: 2.7.1
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3-Clause
Location: /root/miniconda3/envs/deepseek/lib/python3.13/site-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-cufile-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-cusparselt-cu12, nvidia-nccl-cu12, nvidia-nvjitlink-cu12, nvidia-nvtx-cu12, setuptools, sympy, triton, typing-extensions
Required-by: accelerate, bitsandbytes, cut-cross-entropy, peft, timm, torchvision, unsloth, unsloth_zoo, xformers


In [2]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/root/miniconda3/envs/deepseek/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [3]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model" 
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B" 
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit" 

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
# model, tokenizer = FastLanguageModel.from_pretrained(
#     model_name = model_name,
#     max_seq_length = max_seq_length,
#     dtype = dtype,
#     load_in_4bit = load_in_4bit,
# )
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./DeepSeek-R1-Qwen-1.5B-4bit",  # 本地目录
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    local_files_only = True
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.581 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
from peft import PeftModel

In [5]:
adapter_dir = "qwen-1.5b_lora_model"    # 保存的 lora 目录
base_model=model
# 使用 PEFT 将 adapter 加载到 base_model 上
model = PeftModel.from_pretrained(model, adapter_dir)

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [7]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [8]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [9]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [10]:
print(response[0].split("### Response:")[1])


<think>
首先，你提到他是一名28岁的程序员，工作到深夜，感觉头晕、脖子疼，还 sometimes恶心。这些症状让我想到他可能有脑损伤，特别是脑部的损伤可能比较常见。常见的脑损伤有脑膜炎、脑膜炎性脑炎、脑膜炎性脑膜炎以及脑膜炎性脑膜炎等。

接下来，结合他最近的工作表现，工作到深夜很疲劳，说明可能有脑部的损伤导致的疲劳。脑膜炎通常会导致这种症状，而脑膜炎性脑膜炎则可能有更严重的症状。

然后，考虑他最近工作到深夜的情况，这可能与脑膜炎有关。脑膜炎可以导致疲劳、恶心、头痛和头晕，这些症状正是你提到的。

再想想脑膜炎的类型，脑膜炎分为急性和慢性两种，急性脑膜炎可能有更严重的症状，而慢性脑膜炎则可能更稳定。这取决于他的具体病情和治疗情况。

此外，脑膜炎性脑膜炎是急性脑膜炎的一种，它通常与急性脑膜炎有关，可能会有更严重的症状。所以结合他的症状和工作表现，脑膜炎性脑膜炎可能更符合他的描述。

还有其他脑膜炎的类型，比如急性脑膜炎和急性脑膜炎性脑膜炎，这两种都可能导致和你提到的类似症状。急性脑膜炎通常会导致更严重的症状，而急性脑膜炎性脑膜炎可能更稳定。

所以综合来看，脑膜炎和脑膜炎性脑膜炎都是比较符合你提到的症状的。因此，结合这些信息，我认为脑膜炎性脑膜炎可能是你提到的这种情况，因此选择这个选项比较合适。
</think>
根据你描述的症状和症状，男28岁程序员最近工作到深夜，感觉头晕、脖子疼，还偶尔恶心，这些症状提示可能有脑部损伤。通常，脑膜炎和脑膜炎性脑膜炎是常见的脑部损伤类型。脑膜炎可能导致这些症状，而脑膜炎性脑膜炎则可能有更严重的症状。结合症状和工作表现，脑膜炎性脑膜炎可能是你提到的这种情况，因此选择这个选项比较合适。



### 9. 测试训练后的生成结果

In [11]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [12]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
首先，病人已经诊断为急性阑尾炎，这通常会导致腹痛、发热、压痛包块等症状。5天后，病人出现轻度的腹痛和发热，但压痛包块仍然存在。

根据临床实践，病人出现压痛包块可能需要立即处理，以防止感染扩散和降低感染风险。

在急性阑尾炎的早期，可以尝试通过药物治疗来缓解症状。比如，使用抗炎药物，如阿司匹林，以减少炎症反应。

同时，考虑进行无创治疗，如手术或药物治疗，以帮助减轻症状并防止感染。

这些措施可以帮助病人减轻症状，但需要根据具体病史和病人的反应来调整治疗方案。

在处理压痛包块之前，病人最好尽快进行一些药物治疗，以防止感染扩散和降低感染风险。

对于急性阑尾炎患者，及时处理压痛包块可以有效减少感染的发生，同时也能减轻患者的症状和并发症的风险。

因此，针对病人当前的症状，应立即开始药物治疗和无创治疗，并在必要时进行更深入的评估和治疗。
</think>
根据病人的情况，急性阑尾炎的早期症状包括腹痛、发热和压痛包块。由于病人已经发病5天，症状已经轻微减轻，但压痛包块仍然存在，这需要立即采取措施以防止感染扩散。

在急性阑尾炎中，药物治疗可以是有效的方法。例如，使用抗炎药物如阿司匹林来缓解炎症反应，同时可以进行无创治疗如手术来减轻症状。根据病史和反应，患者应立即开始药物治疗和无创治疗，并在必要时进行更深入的评估和治疗。

在处理压痛包块之前，病人最好尽快进行一些药物治疗以防止感染扩散和降低感染风险。



In [13]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )
    
    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [14]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这位60岁男性患者，有右侧胸疼，X线检查显示肋膈角消失，这让我想到可能有肺结核的可能。肺结核通常会带来胸腔积液，也就是肺结核的常见症状之一。

那么，想了解这个胸腔积液的性质，应该怎么做呢？嗯，实验室检查是关键。听说有几种检查可以用来判断胸腔积液的性质。

首先，血常规检查是肯定的。血常规能帮助我们了解血细胞数量，如果血细胞减少，特别是红细胞，那说明有肺结核的可能，因为肺结核会导致低血容量。

再来看尿常规，如果尿液中出现血色和尿素氮，就说明有血尿，这也和肺结核的特征相符。

至于血浆培养，血浆中的蛋白质含量是关键。肺结核会导致血浆中的蛋白质增加，所以血浆培养也很重要，可以帮助我们判断是否有血浆蛋白。

血浆培养结果还能帮助我们判断是肺结核还是其他类型的胸腔积液。比如，如果血浆蛋白增高，但没有血尿，那可能是肺结核；而如果血浆蛋白增高且血尿出现，那就可能是其他情况。

嗯，这样看来，血浆培养是了解胸腔积液性质的最直接方法。因为通过血浆培养，我们能更准确地知道胸腔积液的类型和性质，这对诊断和治疗都很重要。
</think>
对于60岁男性患者，出现右侧胸疼且X线检查显示右侧肋膈角消失，这提示有肺结核的可能。为了更好地了解胸腔积液的性质，进行血浆培养是关键。血浆培养可以提供血浆蛋白水平，帮助诊断肺结核，因为肺结核会导致血浆蛋白增高。如果血浆蛋白增高且血尿出现，可以判断为肺结核；如果血浆蛋白增高但没有血尿，可能是其他类型的胸腔积液。因此，血浆培养是了解胸腔积液性质的最直接方法。


In [45]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这位28岁的男性患者，工作是程序员，长期熬夜，最近突然感觉头晕目眩，甚至有点恶心，这让我想到他可能得了脑损伤性晕眩症。

脑损伤性晕眩症是一种常见的神经系统疾病，尤其是脑损伤后，导致脑缺氧和缺血，从而引起晕眩等症状。这种疾病通常表现为脑缺氧，进而导致晕眩，尤其在长时间的脑损伤后，晕眩症状会更加明显。

再仔细想想，这位患者长时间熬夜，这可能是因为脑缺氧导致的疲劳，甚至更可能导致脑损伤性晕眩。这种情况下，晕眩不仅仅是简单的疲劳，而是与脑缺氧和缺血直接相关。

脑损伤性晕眩症的特征包括脑缺氧、缺血、缺氧血流量和脑缺氧血流量，这些都是导致晕眩的重要原因。这种疾病也常伴随脑水肿和脑损伤，所以患者的情况可能和脑水肿有关。

不过，脑损伤性晕眩症的病因其实不是单一的，它不仅受脑损伤的影响，还可能受到脑损伤前脑水肿的影响。这种情况下，患者可能需要考虑脑水肿作为可能的病因。

总的来说，这位患者可能得了脑损伤性晕眩症，因为他的症状和病因都符合脑损伤后出现的典型表现。不过，由于病因复杂，建议他尽快去看脑水肿专家，以排除脑水肿的可能，从而更好地诊断和治疗。
</think>
```
根据患者的症状和病因分析，这位28岁的男性患者，工作是程序员，长期熬夜，最近突然感觉头晕目眩，甚至有点恶心，这提示他可能得了脑损伤性晕眩症。脑损伤性晕眩症是一种常见的神经系统疾病，尤其是脑损伤后，导致脑缺氧和缺血，从而引起晕眩。这种疾病通常表现为脑缺氧，进而导致晕眩，尤其在长时间的脑损伤后，晕眩症状会更加明显。

此外，脑损伤性晕眩症的病因也受脑损伤前脑水肿的影响，这种情况下，患者可能需要考虑脑水肿作为可能的病因。因此，建议尽快查看脑水肿专家，以排除脑水肿的可能，从而更好地诊断和治疗。

综上所述，这位患者可能得了脑损伤性晕眩症，但建议尽快寻求专业诊断，以确保其病情得到及时准确的治疗。
